In [1]:
from os import environ

# environ['HF_HOME'] = '/Volumes/DATA/hf_home'
environ["HTTP_PROXY"] = "http://localhost:7890"
environ["HTTPS_PROXY"] = "http://localhost:7890"
environ["OPENAI_API_BASE"] = "http://10.1.3.28:8990/v1"
environ["OPENAI_API_KEY"] = "sk-"

from retrieval_qa_benchmark.models import *
from retrieval_qa_benchmark.datasets import *
from retrieval_qa_benchmark.transforms import *
from retrieval_qa_benchmark.utils.registry import REGISTRY

print(str(REGISTRY))

/Users/fangruil/Documents/MQDB/Retrieval-QA-Benchmark/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Datasets:
	hf_dataset
	hotpot_qa
	mmlu
	wikihop
ExtraTransforms:
	base
	mcsa_prompt
Models:
	gpt35
	chatgpt35
Evaluators:



In [ ]:
import yaml

config = yaml.safe_load(open("../config/mmlu-llama2-remote.yaml"))
from retrieval_qa_benchmark.utils.factory import (
    DatasetFactory,
    ModelFactory,
    TransformChainFactory,
    TransformFactory,
)
from retrieval_qa_benchmark.utils.registry import REGISTRY

print(str(REGISTRY))

# yaml factory
dataset = DatasetFactory.from_config(config["dataset"]).build()
transform = TransformChainFactory(
    chain_config=[TransformFactory.from_config(c) for c in config["transforms"]]
).build()
model = ModelFactory.from_config(config["model"]).build()

In [ ]:
print(dataset[0].question)
from retrieval_qa_benchmark.models.openai import GPT, ChatGPT

model = ChatGPT.build(model_name="model")
model.generate(dataset[0].question, temperature=0)

In [ ]:
import json
from retrieval_qa_benchmark.protocols.base import BaseEvaluator

eval = BaseEvaluator(dataset=dataset, llm=model)
print(eval.model_dump())

In [ ]:
import inspect

module = inspect.getmodulename("retrieval_qa_benchmark.protocols.base")

In [ ]:
module.build()

In [3]:
import yaml

config = yaml.safe_load(open("../config/mmlu-llama2-remote.yaml"))
from retrieval_qa_benchmark.utils.factory import (
    DatasetFactory,
    ModelFactory,
    TransformChainFactory,
    TransformFactory,
)
from retrieval_qa_benchmark.utils.registry import REGISTRY

print(str(REGISTRY))

# yaml factory
dataset = DatasetFactory.from_config(config["dataset"]).build()
transform = TransformChainFactory(
    chain_config=[TransformFactory.from_config(c) for c in config["transforms"]]
).build()
model = ModelFactory.from_config(config["model"]).build()

Datasets:
	hf_dataset
	hotpot_qa
	mmlu
	wikihop
Transforms:
	base
	mcsa_prompt
Models:
	gpt35
	chatgpt35
Evaluators:

chain=[MultipleChoiceTransform(sep_chr='\n', prompt_prefix='Please answer with the letter of the correct answer.\n', prompt_suffix='')]


In [2]:
model

ChatGPT(model_name='gpt-3.5-turbo', run_args={'temperature': 0.7, 'top_p': 1.0}, system_prompt='You are a helpful assistant.')